In [ ]:
import sys
from importlib import reload

import polars as pl

sys.path.append("..")

import util

reload(util)

In [ ]:
data = util.load_examples()
for r in data.filter(pl.col("tokens").list.contains("<")).iter_rows(named=True):
    print("".join(r["tokens"]))
    print(r["tags"], "\n\n")


In [3]:
brop = ["(", "[", "{"]
brcl = [")", "]", "}"]


def repl(token: str, tag: str):
    if tag != "br":
        return tag
    if token in brop:
        return "brop"
    elif token in brcl:
        return "brcl"


def repl_row(row):
    tags_new = []
    for token, tag in zip(*row):
        tags_new.append(repl(token, tag))
    return tags_new


tags_new = []
for r in data.select("tokens", "tags").iter_rows():
    tags_new.append(repl_row(r))

data = data.with_columns(pl.Series("tags", tags_new))

In [ ]:
import json


new_data_dict = {
    d["name"]: {
        "difficulty": d["difficulty"],
        "tokens": d["tokens"],
        "tags": d["tags"],
    }
    for d in data.rows(named=True)
}
# with open("../data/examples_annot_2.json", "w", encoding="utf-8") as f:
#     json.dump(new_data_dict, f)
